Calcular la antigüedad promedio de las app de categoría “Family”.

# Cargo el csv y hago ajustes necesarios.

In [ ]:
! pip install pyspark
! pip install -U -q PyDrive
! apt update
! apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=6fb5a545b0d2ad05eede5cd92d19970005cee9756c799be54114ffa99ede88b1
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd
from pyspark.sql.types import IntegerType
from datetime import datetime

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({"id" : "1-oPVltZP-8bpOqQtlRLD6yy6TNasoHLP"})
downloaded.GetContentFile("googleplaystore.csv")

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

Skipeo una línea con el nombre de la aplicación respectivo porque está mal parseada.

In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.read.option("delimiter", ",").option("escape", '"').csv("googleplaystore.csv", header = True, inferSchema = True)
rdd = df.rdd.filter(lambda x: x.App != "Life Made WI-Fi Touchscreen Photo Frame")

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


# Comienzo del ejercicio.

Uso take en cada celda para la revisión y sé que no es necesario.

In [ ]:
rdd_family = rdd.filter(lambda x: x.Category == "FAMILY")
rdd_family.take(5)

[Row(App='YouTube Kids', Category='FAMILY', Rating=4.5, Reviews='470694', Size='Varies with device', Installs='50,000,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Entertainment;Music & Video', Last Updated='August 3, 2018', Current Ver='3.43.3', Android Ver='4.1 and up'),
 Row(App='Candy Bomb', Category='FAMILY', Rating=4.4, Reviews='42145', Size='20M', Installs='10,000,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Casual;Brain Games', Last Updated='July 4, 2018', Current Ver='2.9.3181', Android Ver='4.0.3 and up'),
 Row(App='ROBLOX', Category='FAMILY', Rating=4.5, Reviews='4449910', Size='67M', Installs='100,000,000+', Type='Free', Price='0', Content Rating='Everyone 10+', Genres='Adventure;Action & Adventure', Last Updated='July 31, 2018', Current Ver='2.347.225742', Android Ver='4.1 and up'),
 Row(App='Jewels Crush- Match 3 Puzzle', Category='FAMILY', Rating=4.4, Reviews='14774', Size='19M', Installs='1,000,000+', Type='Free', Price='0', Cont

In [ ]:
rdd_family = rdd_family.map(lambda x: (x.App, datetime.strptime(x["Last Updated"], "%B %d, %Y")))
rdd_family.take(5)

[('YouTube Kids', datetime.datetime(2018, 8, 3, 0, 0)),
 ('Candy Bomb', datetime.datetime(2018, 7, 4, 0, 0)),
 ('ROBLOX', datetime.datetime(2018, 7, 31, 0, 0)),
 ('Jewels Crush- Match 3 Puzzle', datetime.datetime(2018, 7, 23, 0, 0)),
 ('Coloring & Learn', datetime.datetime(2018, 7, 17, 0, 0))]

Convierto las fechas en días (Unix Time Stamp) para facilitarme los cálculos.

In [ ]:
rdd_family = rdd_family.map(lambda x: (x[0], (datetime.now() - x[1]).days)).cache()
rdd_family.take(5)

[('YouTube Kids', 1890),
 ('Candy Bomb', 1920),
 ('ROBLOX', 1893),
 ('Jewels Crush- Match 3 Puzzle', 1901),
 ('Coloring & Learn', 1907)]

La antiguedad promedio en días de las aplicaciones de categoría "Family" es:

In [ ]:
antiguedad_promedio = (rdd_family.map(lambda x: x[1]).reduce(lambda x, y: x + y)) / (rdd_family.count())
antiguedad_promedio

2207.298681541582